# Predict Future Sales

Final project for "How to win a Data Science Competition"

* https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data
* https://www.coursera.org/learn/competitive-data-science/home/welcome

In [26]:
# Importació de mòduls

import numpy as np
import pandas as pd

In [27]:
# Importació de dades

sales = pd.read_csv('competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('competitive-data-science-predict-future-sales/items.csv')
test = pd.read_csv('competitive-data-science-predict-future-sales/test.csv')
categories = pd.read_csv('competitive-data-science-predict-future-sales/item_categories.csv')

## Categories

In [28]:
def category_name2(s):
    # Torna la segona part del nom de la categoria, quan és possible
    s = s.split(' - ')
    if len(s)==2:
        return s[1]
    else:
        return s[0]

In [29]:
# Obtenim subcategories

categories['cat_name1'] = categories.iloc[:,0].apply(lambda s: s.split(' - ')[0])
categories['cat_name2'] = categories.iloc[:,0].apply(category_name2)

In [30]:
categories.head()

,item_category_name,item_category_id,cat_name1,cat_name2
0,PC - Гарнитуры/Наушники,0,PC,Гарнитуры/Наушники
1,Аксессуары - PS2,1,Аксессуары,PS2
2,Аксессуары - PS3,2,Аксессуары,PS3
3,Аксессуары - PS4,3,Аксессуары,PS4
4,Аксессуары - PSP,4,Аксессуары,PSP


## Sales

In [31]:
# Outliers

sales = sales.loc[sales.item_price<10000,:]
sales = sales.loc[sales.item_cnt_day<1500,:]

In [32]:
# Preus negatius

sales = sales.loc[sales.item_price>0,:]

In [33]:
# Devolucions

sales = sales.loc[sales.item_cnt_day > 0,:]

In [34]:
# Ingressos

sales['income'] = sales['item_price'] * sales['item_cnt_day'] 

In [35]:
# Agregació de ventes per mes

sales_month = sales\
            .groupby(['date_block_num','item_id','shop_id'], as_index = False)\
            .agg({'item_cnt_day':'sum', 'income': 'sum', 'item_price': 'mean'})\
            .rename(columns= {'item_cnt_day':'item_cnt'})

In [36]:
sales_month['itemshop_id'] = sales_month['item_id'] * 100 + sales_month['shop_id']

In [37]:
sales_month.head()

,date_block_num,item_id,shop_id,item_cnt,income,item_price,itemshop_id
0,0,19,25,1.0,28.0,28.0,1925
1,0,27,1,1.0,1890.0,1890.0,2701
2,0,27,2,1.0,2499.0,2499.0,2702
3,0,27,10,1.0,1890.0,1890.0,2710
4,0,27,19,1.0,2499.0,2499.0,2719


## Train dataset

In [38]:
# Generem conjunt train: shop, item, date_block_num

train = test.iloc[:,1:].copy()
train['date_block_num'] = 0

for d in range(1,34):
    df = test.iloc[:,1:].copy()
    df['date_block_num'] = d
    train = train.append(df)

In [39]:
# Afegim dies del mes

month_days = [31,28,31,30,31,30,31,31,30,31,30,31]
train['month_days'] = train['date_block_num'].apply(lambda x: month_days[x%12])

In [40]:
# Afegim categoria

train = pd.merge(train, items[['item_id','item_category_id']], how='left', on='item_id')

In [41]:
# Afegim subcategories

train = pd.merge(train, categories[['item_category_id','cat_name1','cat_name2']], how='left', on='item_category_id')

In [42]:
# Afegim ventes

train = pd.merge(train, sales_month, how='left', on=['item_id','shop_id','date_block_num'])

train.fillna(value=0, inplace=True)

In [43]:
# Atribut combinat itemshop

train['itemshop_id'] = train['item_id'] * 100 + train['shop_id']

In [ ]:
train.head()

In [19]:
# TEST

test['month_days'] = 30
test['date_block_num'] = 34

test = pd.merge(test, items[['item_id','item_category_id']], how='left', on='item_id')
test = pd.merge(test, categories[['item_category_id','cat_name1','cat_name2']], how='left', on='item_category_id')

test['itemshop_id'] = test['item_id'] * 100 + test['shop_id']


In [20]:
del sales, categories, sales_month, items

## Conjunt de validació

In [21]:
# Finestra temporal en mesos
WINDOW = 6

# Mes de validació
EVAL_BLOCK = 33

# Mes d'inici de l'entrenament
FIRST_MONTH = EVAL_BLOCK - WINDOW

In [22]:
# Conjunt d'entrenament i validació

train_sales = train.loc[(train['date_block_num']>=FIRST_MONTH)&(train['date_block_num']<EVAL_BLOCK),:]
train_eval = train.loc[train['date_block_num']==EVAL_BLOCK,:]

In [44]:
# Atributs categòrics

cat_feats = ['itemshop_id','item_id','shop_id','item_category_id','cat_name1','cat_name2']

In [24]:
# Frequency encoding: item_cnt

for attrib in cat_feats:
    cnt = train_sales.groupby([attrib], as_index=False)\
                        .agg({'item_cnt':'sum'})\
                        .rename(columns={'item_cnt': (attrib + '_freq')})

    train_sales = pd.merge(train_sales, cnt, how='left', on=attrib)
    train_eval = pd.merge(train_eval, cnt, how='left', on=attrib)

In [25]:
# Eliminem atributs

train_sales.drop(cat_feats, 1, inplace=True)
train_eval.drop(cat_feats, 1, inplace=True)

train_sales.drop('income', 1, inplace=True)
train_eval.drop('income', 1, inplace=True)

In [26]:
train_sales.head()

,date_block_num,month_days,item_cnt,itemshop_id_freq,item_id_freq,shop_id_freq,item_category_id_freq,cat_name1_freq,cat_name2_freq
0,27,30,0.0,7.0,483.0,5772.0,19265.0,74843.0,19881.0
1,27,30,0.0,0.0,0.0,5772.0,32701.0,36913.0,32701.0
2,27,30,0.0,9.0,446.0,5772.0,19265.0,74843.0,19881.0
3,27,30,0.0,1.0,112.0,5772.0,15865.0,74843.0,17339.0
4,27,30,0.0,0.0,0.0,5772.0,30153.0,74843.0,34484.0


In [27]:
# Exportem les dades a CSV

train_sales.to_csv('train_sales.csv', index=False)
train_eval.to_csv('train_eval.csv', index=False)

In [28]:
del train_sales, train_eval

## Preprocessament per a la predicció

In [29]:
# TRAIN

TEST_BLOCK = EVAL_BLOCK + 1
FIRST_MONTH = TEST_BLOCK - WINDOW

# Finestra temporal
train = train.loc[train['date_block_num']>=FIRST_MONTH,:]

# Frequency encoding: item_cnt
for attrib in cat_feats:
    cnt = train.groupby([attrib], as_index=False)\
                        .agg({'item_cnt':'sum'})\
                        .rename(columns={'item_cnt': (attrib + '_freq')})

    train = pd.merge(train, cnt, how='left', on=attrib)
    test = pd.merge(test, cnt, how='left', on=attrib)

In [30]:
# Eliminem atributs

train.drop(cat_feats, 1, inplace=True)
test.drop(cat_feats, 1, inplace=True)

train.drop('income', 1, inplace=True)

In [31]:
# Exportem CSV

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)